# Nash Fixed Point Network - Toy Traffic Routing Problem

This problem illustrates a basic version of traffic routing between 4 nodes. Here we assume all traffic starts at Node 1 and ends at Node 4. Contextual data $d$ is used here to represent "weather." The final cell of this notebook outputs data for an illustration like the following. The left hand side illustrates the context (e.g. sunny and rainy weather). The right side shows the resulting traffic flow and utilization of each edge/road. Darker red edges indicates a higher utilization and, thus, greater commute time. Notice no traffic travels between Node 2 and Node 3 on sunny days, but that traffic is routed along this edge on rainy days. This is consistent with the intuition that there is no need to take backstreets unless the main road is congested.

![notebook-result-illustration](https://drive.google.com/uc?export=view&id=1mz3jLc510pMsNCl1OPQIqLbtyg6TfdeW)

In [1]:
import numpy as np
import torch
import torch.nn               as nn
import torch.distributions as tdist
import sys
import torch.optim            as optim
import torchvision.transforms as transforms 
from torch.utils.data         import Dataset, TensorDataset, DataLoader
from torchvision              import datasets
from torch.optim.lr_scheduler import StepLR
from torch.utils.data         import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

device = "cuda:0"
seed = 42
torch.manual_seed(seed)

#----------------------------------------------------------------------------
#   Graph Structure with Edge Labels
#----------------------------------------------------------------------------
#
#   (1) - 3 - (2) 
#       \      |  \
#         1    4    5
#           \  |      \
#             (3)- 2 - (4)
#
#----------------------------------------------------------------------------
# Global Variables
#----------------------------------------------------------------------------
capacity   = torch.tensor
context    = torch.tensor
action     = torch.tensor
edge       = torch.tensor
inference  = torch.tensor 
context    = torch.tensor

edge_lengths   = torch.tensor([1.0, 2.0, 1.414236, 1.732051, 1.0]).to(device)
travel_speed   = 1.0
free_flow_time = torch.diag(travel_speed * edge_lengths)
rain_factor    = -10
context_size   = 5
num_edges      = 5
W              = torch.rand(num_edges, context_size).to(device)
W[:,0]         = rain_factor * W[:,0] 
def_cap        = torch.tensor([0.4, 0.8, 0.8, 0.6, 0.3]).double().to(device)
sing_val_tol   = 1.0e-8
N              = torch.tensor([[-1, 0, -1, 0, 0],
                               [ 0, 0,  1,-1,-1],
                               [ 1,-1,  0, 1, 0],
                               [ 0, 1,  0, 0, 1]], 
                              dtype=torch.float64).to(device)
U, S, V             = torch.linalg.svd(N, full_matrices=False) 
S[S < sing_val_tol] = 0
S[S > sing_val_tol] = S[S > sing_val_tol] ** -1.0 
Ndag                = torch.mm(torch.mm(V.t(), torch.diag(S)), U.t()).to(device)
b                   = torch.tensor([[-1], [0],[0], [1]], dtype=float).to(device)

def get_trafix_score(x_pred: action, x_true: action, eps=1.0e-2, 
                     tol=1.0e-3) -> float:
  err           = torch.abs(x_pred - x_true)
  denom         = torch.abs(x_true) + tol
  rel_err       = err.div(denom)
  acc_preds     = rel_err < eps  
  num_samples   = x_true.shape[0]
  num_preds     = x_true.shape[1]
  trafix_score  = 100.0 * sum(sum(acc_preds)) / (num_samples * num_preds) 
  return trafix_score

## Generation of Synthetic Data

In [2]:
def sample_context(num_samples: int) -> context:
    ''' Generate sample context for game, size = [num_samples, 5]'''
    context = tdist.Uniform(torch.zeros(num_samples,5), torch.tensor([0.25]))
    return context.sample().to(device)

def cap(d_cap: context) -> capacity:
    ''' Identify capacity along each edge'''
    d_cap     = d_cap.permute(1,0)
    Wd        = W.mm(d_cap).to(device)
    cap_range = 0.4    
    Wd_clamp  = torch.clamp(Wd, min=-cap_range, max=cap_range).to(device)
    rel_cap   = 1 + Wd_clamp
    caps      = torch.mm(torch.diag(def_cap).float(), rel_cap.float()).to(device)
    return caps.permute(1,0).to(device)

def edge_times(x: action, d: context) -> float:
    times          = torch.zeros(x.shape)
    traffic_factor = 1.0 + (x.div(cap(d)) ** 4) 
    traffic_factor = traffic_factor.permute(1,0).float() 
    times          = free_flow_time.mm(traffic_factor)
    return times.permute(1,0).to(device)

def total_time(x: action, d: context) -> float:
    return sum(edge_times(x, d))

def project_C2(x: action) -> action:
    '''Project onto nonnegative orthant'''
    return torch.clamp(x, min=0).to(device)

def project_C1(x: action) -> action: 
    '''Project onto set of flows satisfying origin/destination requirements
    
       Set is defined by {x : Nx=b} and projection uses pseudo inverse of the
       incidence matrix N.
    '''
    x    = x.permute(1,0).to(device)
    proj = x - torch.mm(Ndag, torch.mm(N,x.double())-b)
    return proj.permute(1,0).to(device)

def T(z: action, d: context, return_violation=False, alpha=1.0e-3) -> action: 
    ''' Fixed point operator for Davis Yin Splitting'''
    x    = project_C1(z).double()  
    y    = project_C2(2 * x.to(device) - z - alpha * edge_times(x, d)).float() 
    Tz   = z + y - x 
    if return_violation:
        d_C2_x = torch.norm(x - torch.clamp(x,min=0))
        return Tz.to(device), d_C2_x
    else:
        return Tz.to(device)

def get_game_solution(d: context, fix_pt_tol=1.0e-10, proj_tol=1.0e-10, 
                      max_steps=5000) -> action:
    batches = d.size()[0]
    z       = torch.zeros(batches, num_edges).to(device)
    z_prev  = torch.zeros(batches, num_edges).to(device)
    conv    = False
    depth   = 0
    while not conv and depth < max_steps:
        z_prev = z.clone()
        z, err = T(z, d, return_violation=True)
        res    = torch.norm(z-z_prev)
        depth += 1
        conv   = res < fxd_pt_tol and err < proj_tol 
    x = project_C1(z)
    return torch.clamp(x, min=0)

Generate synthetic training and testing data

In [3]:
def create_data(train_batch_size=20, test_batch_size=200, 
                train_size=2000,  test_size=200): 
    d_context = sample_context(train_size + test_size)
    x_true    = get_game_solution(d_context)
    dataset   = TensorDataset(x_true, d_context)  

    train_dataset, test_dataset = random_split(dataset, 
                                               [train_size, test_size]) 
    train_loader  = DataLoader(dataset=train_dataset,  
                               batch_size=train_batch_size, shuffle=True) 
    test_loader   = DataLoader(dataset=test_dataset,   
                               batch_size=test_batch_size,  shuffle=False) 
    return train_loader, test_loader

Define a Variational Inequality Network for Traffic Routing

In [4]:
class Traffic_Net(nn.Module):
    ''' Nash Fixed Point Network

        For batches, output x is assumed to have size
        [ num_samples, action_size ].
    '''
    def __init__(self, N, b, context_size):
        super().__init__()
        self._N          = N.to(device)
        self._b          = b.to(device)
        self._eps        = 1.0e-8
        x_size           = N.size()[1]
        U, S, V          = torch.linalg.svd(N, full_matrices=False) 
        S[S < self._eps] = 0
        S[S > self._eps] = S[S > self._eps] ** -1.0 
        self.leaky_relu  = nn.LeakyReLU(0.05)
        self._Ndag = torch.mm(torch.mm(V.t(), torch.diag(S)), U.t()).to(device) 
        self.Fx_fc = nn.Linear(5*x_size,   x_size)
        self.x_fc  = nn.Linear(  x_size, 5*x_size, bias=False)
        self.diag  = nn.Parameter(torch.randn(x_size)) 
        self.d_fcs = nn.ModuleList([nn.Linear(  context_size, 5*context_size), 
                                    nn.Linear(5*context_size, 5*context_size), 
                                    nn.Linear(5*context_size, 5*x_size)])         

    def device(self) -> str:
        return next(self.parameters()).data.device

    def project_C1(self, x: action) -> action: 
        '''Project onto set of flows satisfying OD-pair requirements
        
        Set is defined by {x : Nx=b} and projection uses pseudo inverse of 
        the incidence matrix N. 
        '''
        x    = x.permute(1,0)   
        Nx   = torch.mm(self._N, x.double())
        proj = x - torch.mm(self._Ndag, Nx - self._b)
        return proj.permute(1,0)

    def project_C2(self, x: action) -> action:
        '''Project onto nonnegative orthant'''
        return torch.clamp(x, min=0)

    def F(self, x: action, d: context) -> action:  
        '''Learned cost gradient operator'''       
        for fc in self.d_fcs:
            d = self.leaky_relu(fc(d))    
        x  = self.x_fc(x.float())
        xd = x.mul(d)
        Fx = self.Fx_fc(xd)
        return Fx

    def T(self, z: action, d: context, return_violation=False) -> action: 
        ''' Fixed point operator for Davis Yin Splitting'''        
        x  = self.project_C1(z).double()
        Fx = self.F(x,d)
        y  = self.project_C2(2 * x - z - Fx).double()
        Tz = z + y - x
        if return_violation:
            d_C2_x = torch.norm(x - torch.clamp(x,min=0))
            return Tz, d_C2_x
        else:
            return Tz

    def S(self, z: action) -> inference:
        '''Map fixed point z* = T(z*, d) to inference x = P_C2(P_C1(z*))'''
        return torch.clamp(self.project_C1(z), min=0)

    def forward(self, d: context, eps=1.0e-5, max_depth=250, 
                depth_warning=False) -> inference: 
        '''Fixed Point Iteration

           No gradients are stored during main iteration. In training mode,
           one final application of T is applied with gradients attached.
        '''
        with torch.no_grad():
            self.depth = 0.0
            z = torch.zeros((d.shape[0], self._N.shape[1]), device=self.device())
            z_prev = np.Inf * torch.ones(z.shape, device=self.device())            
            all_samp_conv = False
            while not all_samp_conv and self.depth < max_depth:
                z_prev = z.clone()   
                z = self.T(z,d)
                res_norm = torch.max(torch.norm(z - z_prev, dim=1)) 
                self.depth += 1.0
                all_samp_conv = res_norm <= eps
            
        if self.depth >= max_depth and depth_warning:
            print("\nWarning: Max Depth Reached - Break Forward Loop\n")

        attach_gradients = self.training
        return self.S(self.T(z, d)) if attach_gradients else self.S(z)

## Create the model

In [5]:
model = Traffic_Net(N, b, context_size)
model = model.to(device)
print(model)

Traffic_Net(
  (leaky_relu): LeakyReLU(negative_slope=0.05)
  (Fx_fc): Linear(in_features=25, out_features=5, bias=True)
  (x_fc): Linear(in_features=5, out_features=25, bias=False)
  (d_fcs): ModuleList(
    (0): Linear(in_features=5, out_features=25, bias=True)
    (1): Linear(in_features=25, out_features=25, bias=True)
    (2): Linear(in_features=25, out_features=25, bias=True)
  )
)


## Configure Training

In [6]:
learning_rate = 2e-4
optimizer     = optim.Adam(model.parameters(), lr=learning_rate)
fxd_pt_tol    = 1.0e-3
criterion     = nn.MSELoss()
max_epochs    = 1000
lr_scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=250, gamma=0.5)
fmt  = '[{:4d}/{:4d}]: train loss = {:5.2e} | train TRAFIX = {:6.2f}% | '  
fmt += 'test TRAFIX = {:6.2f}% | depth = {:5.1f} | lr = {:5.1e}'

if not 'train_loader' in globals():
    train_loader, test_loader = create_data()

## Execute Training

In [7]:
print('\nTraining N-FPN')
test_loss    = 0.0
train_loss   = 0.0
train_trafix = 0.0
test_trafix  = 0.0
model.train()

for epoch in range(max_epochs): 
    for x_batch, d_batch in train_loader:  
        optimizer.zero_grad()
        x_pred = model(d_batch, eps=fxd_pt_tol) 
        loss   = criterion(x_pred, x_batch)
        loss.backward()
        optimizer.step()        
        train_loss   += 0.1 * (loss.item() - train_loss)
        train_trafix += 0.1 * (get_trafix_score(x_pred, x_batch) - train_trafix)

    for x_batch, d_batch in test_loader:
        with torch.no_grad():
            x_pred       = model(d_batch, eps=fxd_pt_tol) 
            loss         = criterion(x_pred, x_batch) 
            test_loss   += 0.5 * (loss.item() - test_loss)
            test_trafix += 0.5 * (get_trafix_score(x_pred, x_batch) - test_trafix)

    lr_scheduler.step()
    print(fmt.format(epoch+1, max_epochs, train_loss, train_trafix, 
                     test_trafix, model.depth, 
                     optimizer.param_groups[0]['lr']))


Training N-FPN
[   1/1000]: train loss = 2.74e-01 | train TRAFIX =   0.00% | test TRAFIX =   0.00% | depth =  46.0 | lr = 2.0e-04
[   2/1000]: train loss = 9.60e-02 | train TRAFIX =   0.05% | test TRAFIX =   0.00% | depth =  61.0 | lr = 2.0e-04
[   3/1000]: train loss = 6.78e-03 | train TRAFIX =   2.92% | test TRAFIX =   1.10% | depth =  50.0 | lr = 2.0e-04
[   4/1000]: train loss = 4.82e-03 | train TRAFIX =   3.05% | test TRAFIX =   2.15% | depth =  47.0 | lr = 2.0e-04
[   5/1000]: train loss = 4.41e-03 | train TRAFIX =   4.06% | test TRAFIX =   2.23% | depth =  45.0 | lr = 2.0e-04
[   6/1000]: train loss = 4.11e-03 | train TRAFIX =   3.90% | test TRAFIX =   2.86% | depth =  42.0 | lr = 2.0e-04
[   7/1000]: train loss = 4.24e-03 | train TRAFIX =   3.30% | test TRAFIX =   2.88% | depth =  43.0 | lr = 2.0e-04
[   8/1000]: train loss = 3.72e-03 | train TRAFIX =   4.44% | test TRAFIX =   2.79% | depth =  43.0 | lr = 2.0e-04
[   9/1000]: train loss = 3.26e-03 | train TRAFIX =   3.59% | te

## Generate Ground Truth and Predictions for N-FPN Toy Figures

Below we hand choose two contexts. The first has a large value in the first component, corresponding to a rainy day (see the definition of W above, which has a negative value in first column) and a sunny day second.

_Note:_ Some quantites are scaled so that they may be directly copied into Tikz in LaTex to generate each of the 4 subfigures.

In [8]:
d_fig = torch.tensor([[0.2, 0.15, 0.15, 0.1, 0.15], 
                      [0.05, 0.2, 0.2, 0.25, 0.25]]).to(device)
 
x_figures = get_game_solution(d_fig).detach().cpu() # .numpy()
caps_figures = cap(d_fig)
caps_figures = caps_figures.detach().cpu().numpy()

print('Rainy Day Caps =\n', 0.5 * caps_figures[0,:])
print('Sunny Day Caps =\n', 0.5 * caps_figures[1,:])

print('\n-------   True    ---------\n')
print('Rainy Day Traffic =\n', x_figures[0,:])
print('Sunny Day Traffic =\n', x_figures[1,:])

print('Rainy Day Utilization =\n', 40 * x_figures[0,:] / caps_figures[0,:])
print('Sunny Day Utilization =\n', 40 * x_figures[1,:] / caps_figures[1,:]) 

print('\n------- Predicted ---------\n')
x_pred = model(d_fig)
x_pred = x_pred.detach().cpu() 

print('Rainy Day Traffic =\n', x_pred[0,:])
print('Sunny Day Traffic =\n', x_pred[1,:])

print('Rainy Day Utilization (scaled) =\n', 40 * x_pred[0,:] / caps_figures[0,:])
print('Sunny Day Utilization (scaled) =\n', 40 * x_pred[1,:] / caps_figures[1,:]) 
    
print('\n\nIllustration TRAFIX score = {:3.2f}%'.format(get_trafix_score(x_pred, x_figures)))

Rainy Day Caps =
 [0.12       0.24000001 0.24000001 0.18       0.10056677]
Sunny Day Caps =
 [0.23117547 0.47123381 0.46100006 0.39020237 0.18706368]

-------   True    ---------

Rainy Day Traffic =
 tensor([0.3831, 0.6448, 0.6169, 0.2617, 0.3552], dtype=torch.float64)
Sunny Day Traffic =
 tensor([0.5316, 0.5316, 0.4684, 0.0000, 0.4684], dtype=torch.float64)
Rainy Day Utilization =
 tensor([63.8433, 53.7321, 51.4117, 29.0806, 70.6426], dtype=torch.float64)
Sunny Day Utilization =
 tensor([45.9868, 22.5600, 20.3232,  0.0000, 50.0844], dtype=torch.float64)

------- Predicted ---------

Rainy Day Traffic =
 tensor([0.3835, 0.6460, 0.6165, 0.2625, 0.3540], dtype=torch.float64)
Sunny Day Traffic =
 tensor([5.3266e-01, 5.3266e-01, 4.6734e-01, 1.2913e-06, 4.6734e-01],
       dtype=torch.float64)
Rainy Day Utilization (scaled) =
 tensor([63.9138, 53.8349, 51.3764, 29.1706, 70.3973], dtype=torch.float64)
Sunny Day Utilization (scaled) =
 tensor([4.6083e+01, 2.2607e+01, 2.0275e+01, 6.6186e-05, 